---

### **Chapter 5-37**

#### **내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라.**


- 딕셔너리, 긴 튜플, 다른 내장 타입이 복잡하게 내포된 데이터를 값으로 사용하는 딕셔너리를 만들지 말라.


In [2]:
# 딕셔너리 안에 딕셔너리, 그 안에 또 리스트가 들어가는 구조(Deeply nested)는 코드를 읽기 어렵게 함

# 피해야 할 구조: 딕셔너리 안의 딕셔너리 안의 리스트
# data['기관']['부서']['인원'][0][2] <- '2'가 무엇을 의미하는지 알기 어려움
organization_data = {
    'group_a': {
        'dept_01': {
            'members': [('User1', 20, 'Level_A'), ('User2', 25, 'Level_B')]
        }
    }
}

print(organization_data)

{'group_a': {'dept_01': {'members': [('User1', 20, 'Level_A'), ('User2', 25, 'Level_B')]}}}


- 완전한 클래스가 제공하는 유연성이 필요하지 않고 가벼운 불변 데이터 컨테이너가 필요하다면 `namedtuple`을 사용하라


In [3]:
# namedtuple : 데이터의 의미를 명확히 하면서도 가볍게 유지하고 싶을 때 사용합니다.

from collections import namedtuple

# 각 필드에 이름을 부여하여 가독성 확보
Member = namedtuple('Member', ['id', 'age', 'rank'])

new_member = Member(id='User1', age=20, rank='Level_A')
print(new_member.id)   # 인덱스 대신 이름(.id)으로 접근 가능
print(new_member.rank) # 훨씬 직관적임

User1
Level_A


- 내부 상태를 표현하는 딕셔너리가 복잡해지면 이 데이터를 관리하는 코드를 여러 클래스로 나눠서 재작성하라.

In [4]:
# 상태 관리 로직이 복잡해지면 각 계층을 클래스로 나누는 것이 유지보수에 유리함.

class RankRecord:
    def __init__(self):
        self._history = []
    def add_record(self, score):
        self._history.append(score)

class MemberManager:
    def __init__(self):
        self._members = {}
    def get_member(self, name):
        if name not in self._members:
            self._members[name] = RankRecord()
        return self._members[name]

---

### **Chapter 5-38**

#### **간단한 인터페이스의 경우 클래스 대신 함수를 받아라.**


- 파이썬의 여러 컴포넌트 사이에 간단한 인터페이스가 필요할 때는 클래스를 정의하고 인스턴스화하는 대신 간단히 함수를 사용할 수 있음.
- 파이썬 함수나 메서드는 일급 시민이다. 따라서 (다른타입의 값과 마찬가지로) 함수나 함수 참조를 식에 사용할 수 있다.



In [ ]:
# 파이썬의 함수는 객체(일급 시민)이므로,
# API가 '실행 가능한 동작'을 요구할 때 굳이 복잡한 클래스를 만들 필요 없이 함수 자체를 넘기면 됨.

names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']

# 굳이 길이를 구하는 클래스를 정의할 필요가 없음
# def get_len(name): return len(name) 처럼 함수만 정의하면 됨

# len 함수 자체를 인자로 전달 (일급 시민)
names.sort(key=len) 

print(names)
# 결과: ['Plato', 'Socrates', 'Aristotle', 'Archimedes']

['Plato', 'Socrates', 'Aristotle', 'Archimedes']



- `__call__`특별 메서드를 사용하면 클래스의 인스턴스인 객체를 일반 파이선 함수처럼 호출할 수 있다. 


In [ ]:
# 클래스의 인스턴스이지만, 문법적으로는 함수처럼 ()를 붙여 실행할 수 있게 함.

class BetterRepeater:
    def __init__(self, message):
        self.message = message
    
    def __call__(self, times):      # 이 메서드 덕분에 인스턴스를 함수처럼 호출 가능
        return (self.message + " ") * times

# 인스턴스 생성
repeat_hello = BetterRepeater("Hello")

# 함수처럼 호출! (실제로는 repeat_hello.__call__(3)이 실행됨)
print(repeat_hello(3)) 
# 결과: Hello Hello Hello

- 상태를 유지하기 위한 함수가 필요한 경우에는 상태가 있는 클로저를 정의하는 대신 `__call__`메서드가 있는 클래스를 정의할지 고려해보라


In [ ]:
# 클로저
def make_counter():
    count = 0  # 상태(State)
    
    def counter():
        nonlocal count # 상태를 변경하기 위해 nonlocal 선언 필요
        count += 1
        return count
        
    return counter

my_counter = make_counter()
print(my_counter()) # 1
print(my_counter()) # 2
# 상태(count)를 밖에서 확인하거나 초기화하기 어려움

1
2


In [ ]:
# __call__ 메서드
class CountMissing:
    def __init__(self):
        self.added = 0  # 상태(State)가 명확함
        
    def __call__(self):
        self.added += 1
        return 0
        
# 사용 예시: defaultdict의 기본값 생성기로 사용
from collections import defaultdict

counter = CountMissing()
current = {'green': 12, 'blue': 3}
increments = [('red', 5), ('blue', 17), ('orange', 9)]
result = defaultdict(counter, current) # 객체(counter)를 함수처럼 전달

for key, amount in increments:
    result[key] += amount

# 결과 확인: 함수처럼 작동했지만, 상태(added)를 나중에 조회 가능
print(f"새로 추가된 키의 개수: {counter.added}") 
# 결과: 새로 추가된 키의 개수: 2 ('red', 'orange'가 없었으므로 __call__이 2번 호출됨)

새로 추가된 키의 개수: 2


---

### **Chapter 5-39**

#### **객체를 제너릭하게 구성하려면 `@classmethod`를 통한 다형성을 활용하라.**


- 파이썬의 클래스에는 생성자가 `__init__` 메서드 뿐이다.
- `@classmetod`를 사용하면 클래스에 다른 생성자를 정의할 수 있다.


In [7]:
# `__init__`은 기본적인 초기화만 담당하고,
# 다양한 입력 형태(예: JSON, 파일, 문자열 등)를 처리하는 로직은 클래스 메서드로 분리.

class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    # __init__ 외의 또 다른 생성자 역할 (Factory Method)
    @classmethod
    def from_json(cls, json_data):
        # cls(...)를 호출함으로써 하위 클래스에서도 호환됨
        return cls(json_data['title'], json_data['author'])

    # 또 다른 생성자 예시
    @classmethod
    def from_string(cls, string_data):
        title, author = string_data.split('-')
        return cls(title, author)

# 사용 예시
book1 = Book("Effective Python", "Brett Slatkin")
book2 = Book.from_json({"title": "Clean Code", "author": "Robert C. Martin"})
book3 = Book.from_string("Refactoring-Martin Fowler")

print(book2.title)  # Clean Code
print(book3.title)
print(book3.author)

Clean Code
Refactoring
Martin Fowler


- 클래스 메서드 다형성을 활용하면 여러 구체적인 하위 클래스의 객체를 만들고 연결하는 제너릭한 방법 제공이 가능하다.


In [9]:
# 1. 공통 인터페이스 (뼈대)
class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

# 2. 구체적인 데이터 클래스 (Input)
class PathInputData(GenericInputData):
    def __init__(self, path):
        self.path = path

    def read(self):
        # 실제 파일 대신 테스트를 위한 가짜 데이터 반환
        return f"Data from {self.path}\nLine 2\nLine 3"

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        # 가상의 파일 경로를 가진 객체 리스트 반환
        return [cls(f"{data_dir}/file1.txt"), cls(f"{data_dir}/file2.txt")]

# 3. 작업자 클래스 (Worker)
class LineCountWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = 0

    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n') + 1

    # [핵심] 제너릭한 연결 메서드 (다형성 활용)
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        # input_class가 무엇이든(PathInputData 등), 그 클래스의 generate_inputs를 호출
        inputs = input_class.generate_inputs(config)
        
        for input_data in inputs:
            workers.append(cls(input_data)) # LineCountWorker 생성 및 연결
        return workers

# 4. 실행 함수 (MapReduce 시뮬레이션)
def map_reduce(worker_class, input_class, config):
    # 구체적인 클래스 이름 없이도 연결 가능
    workers = worker_class.create_workers(input_class, config)
    
    total_lines = 0
    for w in workers:
        w.map()
        total_lines += w.result
        
    return total_lines

# --- 실행 ---
config = {'data_dir': '/tmp/logs'}

# LineCountWorker와 PathInputData를 연결
result = map_reduce(LineCountWorker, PathInputData, config)

print(f"총 줄 수: {result}") 
# 예상 결과: 6 (파일 2개 * 각 3줄)

총 줄 수: 6


---

### **Chapter 5-40**

#### **super로 부모 클래스를 초기화하라.**


- 파이썬은 표준 메서드 결정 순서(MRO)를 활용해 상위 클래스 초기화 순서와 다이아몬드 상속 문제를 해결함
- 부모 클래스를 초기화 할 때는 super내장 함수를 아무 인자 없이 호출하라. super를 아무 인자 없이 호출하면 파이썬 컴파일러가 자동으로 올바른 파라미터를 넣어준다.


##### **super()를 안쓰고 직접 호출할 때**

- 부모 클래스를 Base.__init__(self) 처럼 직접 호출하면, 다이아몬드 구조의 최상단 클래스(Base)가 두 번 초기화되는 심각한 문제가 발생

In [ ]:
class Base:
    def __init__(self):
        print("Base 초기화")

class Left(Base):
    def __init__(self):
        Base.__init__(self) # 직접 호출 (문제 발생 원인)
        print("Left 초기화")

class Right(Base):
    def __init__(self):
        Base.__init__(self) # 직접 호출
        print("Right 초기화")

class Diamond(Left, Right):
    def __init__(self):
        Left.__init__(self)
        Right.__init__(self)
        print("Diamond 초기화")

d = Diamond()  # Diamond 실행 시,
               # (Base init) + Left -> (Base init) + Right + Diamond

Base 초기화
Left 초기화
Base 초기화
Right 초기화
Diamond 초기화


##### **super() 사용(Good)**

- super()는 단순히 "부모 클래스"를 부르는 게 아니라, MRO(메서드 결정 순서)의 다음 순서를 호출
    - 따라서 Base는 딱 한 번만 실행.

In [11]:
class Base:
    def __init__(self):
        print("Base 초기화")

class Left(Base):
    def __init__(self):
        super().__init__() # 인자 없이 호출 (Python이 알아서 처리)
        print("Left 초기화")

class Right(Base):
    def __init__(self):
        super().__init__()
        print("Right 초기화")

class Diamond(Left, Right):
    def __init__(self):
        super().__init__() # MRO 순서에 따라 체인처럼 연결됨
        print("Diamond 초기화")

d = Diamond()
print("\n[MRO 순서 확인]")
print(Diamond.mro())

Base 초기화
Right 초기화
Left 초기화
Diamond 초기화

[MRO 순서 확인]
[<class '__main__.Diamond'>, <class '__main__.Left'>, <class '__main__.Right'>, <class '__main__.Base'>, <class 'object'>]


---

### **Chapter 5-41**

#### **기능을 합성할 때는 믹스인 클래스를 사용하라.**


- 믹스인을 사용해 구현할 수 있는 기능을 인스턴스 애트리뷰트와 `__init__`을 사용하는 다중 상속을 통해 구현하지 말라.
- 믹스인 클래스가 클래스별로 특화된 기능을 필요로 한다면 인스턴스 수준에서 끼워넣을 수 있는 기능(정해진 메서드를 통해 해당 기능을 인스턴스가 제공하게 만듦)을 활용하라.
- 믹스인에는 필요에 따라 인스턴스 메서드는 물론 클래스 메서드도 포함될 수 있다.
- 믹스인을 합성하면 단순한 동작으로부터 더 복잡한 기능을 만들어낼 수 있다.



In [ ]:
import json


# ==============# 1. 기본 믹스인 (원칙 1, 2 적용)==============
class ToDictMixin:
    # [원칙 1] __init__ 없음
    # 상태(인스턴스 변수)를 초기화하지 않고 기능(메서드)만 제공.
    
    def to_dict(self):
        # [원칙 2] 인스턴스 수준의 끼워넣기 (Hook 메서드)
        # 믹스인은 "무엇을" 저장할지 모릅니다. 
        # _get_serialized_keys()를 호출하여 하위 클래스에게 물어봅니다.
        keys = self._get_serialized_keys()
        return {k: getattr(self, k) for k in keys}



# ========= 2. 확장 믹스인 (원칙 3, 4 적용)==========
class JsonMixin(ToDictMixin):  # [원칙 4] 믹스인 합성
    # ToDictMixin을 상속받아 '단순한 dict 변환' 기능을 'JSON 직렬화'라는 기능으로 확장.
    
    def to_json(self):
        return json.dumps(self.to_dict())

    # [원칙 3] 클래스 메서드 포함
    # 믹스인은 인스턴스 메서드뿐만 아니라, 객체 생성을 돕는 클래스 메서드도 가질 수 있음.
    @classmethod
    def from_json(cls, json_str):
        data = json.loads(json_str)
        # cls(**data)를 통해 User 등의 구체적인 객체를 생성
        return cls(**data)


# ===================3. 실제 사용 클래스========================
class User(JsonMixin):
    # 실제 클래스만 __init__을 가짐
    def __init__(self, name, age, password):
        self.name = name
        self.age = age
        self.password = password

    # [원칙 2의 구현] 끼워넣기(Hook) 구현
    # User 클래스만의 특화된 로직 (비밀번호는 직렬화에서 제외)
    def _get_serialized_keys(self):
        return ['name', 'age']

# --- 실행 ---
user = User("James", 25, "secret_code")

# 1. 믹스인 합성 결과 (to_json 사용)
print(f"JSON 변환: {user.to_json()}") 
# 출력: {"name": "Minje", "age": 24}

# 2. 클래스 메서드 사용 (from_json 사용)
user2 = User.from_json('{"name": "Alice", "age": 30, "password": "unknown"}')
print(f"복원된 객체: {user2.name}, {user2.age}")
# 출력: Alice, 30

JSON 변환: {"name": "James", "age": 25}
복원된 객체: Alice, 30


---

### **Chapter 5-42**

#### **비공개 애트리뷰트보다는 공개 애트리뷰트를 사용하라.**




- 파이썬 컴파일러는 비공개 애트리뷰트를 자식 클래스나 클래스 외부에서 사용하지 못하도록 엄격히 금지하지 않는다.
- 여러분의 내부 API에 있는 클래스의 하위 클래스를 정의하는 사람들이 여러분이 제공하는 클래스의 애트리뷰트를 사용하지 못하도록 막기보다는 애트리뷰트를 사용해 더 많은 일을 할 수 있게 허용하라.
- 비공개 애트리뷰트로 (외부나 하위 클래스의) 접근을 막으려고 시도하기보다는 보호된 필드를 사용하면서 문서에 적절한 가이드를 남겨라
- 여러분이 코드 작성을 제어할 수 없는 하위 클래스에서 이름 충돌이 일어나는 경우를 막고 싶을 때만 비공개 애트리뷰트를 사용할 것을 권장함.


##### 1. 공개 애트리뷰트 (Public)
- 모양: 밑줄 없음 (self.name)
- 특징: 클래스 안에서도, 밖에서도 마음대로 쓰고 고칠 수 있음.

##### 2. 보호된 애트리뷰트 (Protected)
- 모양: 밑줄 1개 (self._money)
- 특징: 공개(Public)와 똑같이 접근 가능 (그러나 비권장)

##### 3. 비공개 애트리뷰트 (Private)
- 모양: 줄 2개 (self.__secret)
- 특징: 클래스 밖에서는 이름이 안 보임. (접근 시도 시 에러 발생) <br>
        상속받은 자식 클래스에서도 안 보임. (이름이 겹치는 사고를 막기 위해서)

In [ ]:
# `_` 1개 애트리뷰트

class DBConnection:
    def __init__(self):
        # [Good] '_'(보호된 필드) 사용
        # 의미: "내부용이지만, 하위 클래스에서 필요하면 건드려도 됩니다."
        self._timeout = 30 

    def connect(self):
        print(f"Connecting with timeout: {self._timeout}s")

class FastDBConnection(DBConnection):
    def __init__(self):
        super().__init__()
        # 하위 클래스가 부모의 내부 속성(_timeout)을 자유롭게 수정 가능
        # 만약 부모가 __timeout을 썼다면 여기서 AttributeError 발생
        self._timeout = 5 

conn = FastDBConnection()
conn.connect() 
# 출력: Connecting with timeout: 5s

In [ ]:
# `_` 2개 애트리뷰트

class ApiBase:
    def __init__(self):
        # [Use Case] '__'(비공개 필드) 사용
        # 의미: "이건 정말 나만 써야 해. 자식이 우연히 덮어쓰면 안 돼."
        self.__internal_value = 5  
        # 파이썬은 이를 _ApiBase__internal_value 로 이름을 바꿔버림

    def get_base_value(self):
        return self.__internal_value

class MyApi(ApiBase):
    def __init__(self):
        super().__init__()
        # 실수로 부모와 같은 이름을 사용함!
        # 하지만 부모의 __internal_value와는 서로 다른 변수로 취급됨
        self.__internal_value = 999 

api = MyApi()

# 자식 클래스가 값을 999로 바꿨지만, 부모 클래스의 메서드는 영향받지 않음
print(f"부모의 값: {api.get_base_value()}") 
# 출력: 부모의 값: 5 (안전하게 보호됨)

print(f"자식의 값: {api._MyApi__internal_value}") 
# 출력: 자식의 값: 999

부모의 값: 5
자식의 값: 999


---

### **Chapter 5-43**

#### **커스텀 컨테이너 타입은 `collections.abc`를 상속하라.**


- 간편하게 사용할 경우에는 파이썬 컨테이너 타입(리스트나 딕셔너리 등)을 직접 상속하라.
- 커스텀 컨테이너를 제대로 구현하려면 수많은 메서드를 구현해야 한다는 점에 주의하라.
- 커스텀 컨테이너 타입이 collection.abc에 정의된 인터페이스를 상속하면 커스텀 컨테이너 타입이 정상적으로 작동하기 위해 필요한 인터페이스와 기능을 제대로 구현하도록 보장할 수 있다.


In [15]:
# 기존 리스트의 기능을 그대로 쓰면서 메서드 하나만 추가하고 싶을 때는
#  list를 바로 상속받는 게 가장 빠르고 효율적임.

# 간편하게 사용할 경우 내장 타입(list) 상속
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members) # 기존 리스트 초기화 로직 그대로 재사용

    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

# 사용
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a'])
print(f"길이: {len(foo)}")  # 기존 list 기능 (len) 작동
foo.append('d')             # 기존 list 기능 (append) 작동
print(f"빈도수: {foo.frequency()}") # 추가된 기능
# 출력: 빈도수: {'a': 3, 'b': 2, 'c': 1, 'd': 1}

길이: 6
빈도수: {'a': 3, 'b': 2, 'c': 1, 'd': 1}


In [16]:
# 리스트처럼 동작하는 완전히 새로운 자료구조(예: 트리 구조)를 만들 때는
#  list를 상속받는 것보다 collections.abc를 쓰는 게 좋음

from collections.abc import Sequence

# [Point 3] collections.abc의 Sequence 상속
class BinaryNode(Sequence):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    # Sequence가 요구하는 필수 메서드 1
    def __len__(self):
        # 재귀적으로 전체 노드 개수 계산
        count = 1  # 나 자신
        if self.left: count += len(self.left)
        if self.right: count += len(self.right)
        return count

    # Sequence가 요구하는 필수 메서드 2
    def __getitem__(self, index):
        # 인덱스로 트리 탐색 (구현 생략 - 복잡함 방지)
        if index == 0: return self.value
        # ... (중략: 재귀적으로 인덱스 찾는 로직) ...
        raise IndexError

# 사용
tree = BinaryNode(10, left=BinaryNode(5), right=BinaryNode(15))

# [효과]
# 나는 __len__과 __getitem__만 짰는데,
# Sequence를 상속받은 덕분에 'index'와 'count' 메서드가 공짜로 생김!

# print(tree.index(15))  # <--- 구현 안 했는데도 작동함! (Mixin 효과)
# print(tree.count(10))  # <--- 구현 안 했는데도 작동함!